In [ ]:
!pip install chromadb
import numpy as np
import json
import ast
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import chromadb
import sentence_transformers
from tqdm.autonotebook import tqdm, trange

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
transcription_data = '/content/transcriptions.json'

In [ ]:
import chromadb
from chromadb.config import Settings

client_capstone = chromadb.PersistentClient(path="/content/chromadb")
collection_transcriptions = client_capstone.create_collection(name="Capstone_Kiwi")

In [ ]:
with open(transcription_data, 'r') as file:
    transcription_data = json.load(file)

In [ ]:
data = transcription_data

In [ ]:
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer

In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import CLIPProcessor, CLIPModel
from transformers import CLIPTokenizer, CLIPModel
import chromadb
from chromadb.config import Settings
model_name = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(model_name)
clip_tokenizer = CLIPTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = clip_model.to(device)

In [ ]:
# embeddings = []
from transformers import CLIPProcessor, CLIPModel
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")
def embed_text(text):
    text_inputs = processor(text=[text], return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        text_embeddings = clip_model.get_text_features(**text_inputs)
    return text_embeddings.squeeze().cpu().numpy()
embeddings = []
for item in transcription_data:
    summary_text = item['summary']
    embedding = embed_text(summary_text)
    embeddings.append(embedding)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
import chromadb
from PIL import Image
import numpy as np
from transformers import CLIPProcessor, CLIPModel
import chromadb
from chromadb.config import Settings
import os

client_capstone_5 = chromadb.PersistentClient(path="/content/chromadb2")
collection_images = client_capstone_5.create_collection(name="Capstone_KiwiImagesrPath")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(model_name).to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
import chromadb
import os
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import torch

def get_image_embedding(image_path):
    image = Image.open(image_path)
    inputs = processor(images=image, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        embedding = model.get_image_features(**inputs)
    return embedding.cpu().numpy().flatten()

def add_images_to_collection(image_folder, collection):
    image_embeddings = []
    image_names = []
    image_paths = [os.path.join(image_folder, filename) for filename in os.listdir(image_folder)
                   if os.path.isfile(os.path.join(image_folder, filename)) and filename.endswith(('.png', '.jpg', '.jpeg'))]

    for image_path in image_paths:
        embedding = get_image_embedding(image_path)
        image_name = os.path.basename(image_path)
        image_embeddings.append(embedding)
        image_names.append(image_name)

    collection.add(
        documents=image_names,
        embeddings=image_embeddings,
        metadatas=[{"image_name": name} for name in image_names],
        ids=[str(i) for i in range(len(image_paths))]
    )

image_folder = '/content/drive/MyDrive/scenes'
add_images_to_collection(image_folder, collection_images)


In [ ]:
type(transcription_data)

list

In [ ]:
for i, item in enumerate(transcription_data):
    summary_text = item['summary']
    collection_transcriptions.add(
        documents=str(item),
        embeddings=[embeddings[i]],
        metadatas=None,
        ids=[str(i)]
    )
print("Data successfully added to collection_transcriptions.")


Data successfully added to collection_transcriptions.


In [ ]:
query_text = "Finding the Square Root of a Number"
query_embedding = embed_text(query_text)
results = collection_transcriptions.query(query_embeddings=query_embedding, n_results=3)
for i in range(len(results['documents'])):
    print(f"Document: {results['documents'][i]}")
    print(f"ID: {results['ids'][i]}")
    print(f"Distance: {results['distances'][i]}")


Document: ['{\'start_time\': \'0:14:15\', \'end_time\': \'0:15:00\', \'transcription\': [{\'speaker\': \'A\', \'text\': "And you keep doing this until you decide that you\'re close enough. Okay, so then. So what we saw for the imperative knowledge in the previous numerical example was the recipe. Right. For how to find the square root of x. So what were the three parts to the recipe? One was a simple sequence of steps. Right. There were four steps, pretty much. The other was a flow of control. So there were parts where we made decisions. Are we close enough? There were parts where we repeated some steps. So at the end, we said, repeat steps 1, 2, 3. So that\'s the flow of control. And the last part of the recipe was a way to stop. You don\'t want a program that keeps."}], \'summary\': \'The text explains the process of finding the square root of x through a recipe involving a sequence of steps, decision-making for control flow, and a mechanism to terminate the process.\', \'image_names

In [ ]:
import shutil
chroma_directory = '/content/chroma_db'
shutil.make_archive('/content/chroma_db', 'zip', chroma_directory)
print("Chroma vector store zipped successfully.")
from google.colab import files
files.download('/content/chroma_db.zip')

Chroma vector store zipped successfully.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>